In [334]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [323]:
df = pd.read_excel('Book3.xlsx', sheet_name='Sheet3')
df.head(5)

,State,LGA,Facility,Year,Month,Old_Clients_10-14yrs_male,New_Clients_10-14yrs_male,Safe_Sex_10-14yrs_male,STI_Prevention_10-14yrs_male,Contraceptive_Use_10-14yrs_male,...,Others_25-35yrs_female,Treatment_of_Minor_Ailments_25-35yrs_female,Number_of_Adolescents_and_Young_People_reached_through_health_talks_held_at_the_centre_25-35yrs_female,Number_of_School_Visits_Conducted_25-35yrs_female,Number_of_Adolescents_and_Young_People_reached_through_school_visits_25-35yrs_female,Number_of_Outreaches_/Group_health_talk_conducted_25-35yrs_female,Number_of_Adolescents_and_Young_People_reached_through_outreaches/group_health_talks_conducted_25-35yrs_female,Number_of__In_School_Youth_(ISY)_that_accessed_the_centre_25-35yrs_female,Number_of__Out_School_Youth_(OSY)_that_accessed_the_centre_25-35yrs_female,Number_of_Clients_referred_25-35yrs_female
0,Akwa-Ibom,Akamkpa,cr Akamkpa General Hospital,2021,January,78,180,85,171,85,...,129,171,71,86,20,77,99,47,22,143
1,Akwa-Ibom,Calabar South,cr Akamkpa General Hospital,2022,February,157,134,69,38,188,...,144,149,68,133,121,52,73,83,127,75
2,Akwa-Ibom,Akamkpa,cr Akamkpa General Hospital,2022,March,163,122,41,124,34,...,126,135,64,182,97,146,93,143,91,142
3,Akwa-Ibom,Calabar Municipal,cr Akamkpa General Hospital,2021,April,38,48,29,56,200,...,146,40,29,60,198,37,126,59,179,60
4,Akwa-Ibom,Ikom,cr Akamkpa General Hospital,2021,May,55,59,85,141,73,...,186,80,56,150,44,195,53,55,125,128


In [324]:
body = {
    "Total  Attendance":     ["Old Clients", "New Clients"],
    "Counseling on":            ["Safe Sex", "STI Prevention", "Contraceptive Use",
                                 "Drug Abuse", "Sexual Violence", "Unplanned Pregnancy", "Others (specify)"],
    " HIV Testing and Couseling": ["HIV Result negative (-ve)", "HIV Result positive (+ve)"],

    "Family Planning": ["Total no of clients receiving condom",
                        "Total no of condoms dispensed (in pieces)",
                        "Total no clients receiving Oral Contraceptive Pills (OPC)",
                        "Total no of clients receiving injectables",
                        "Total no of clients that had Implants inserted"],

    "HIV Care and Treatment": ["Enrolment into Care",
                               "HIV Treatment (ART)",
                               "PMTCT"],
    "Other Services": ["Post Abortion Care",
                       "Sydromic Management of STIs",
                       "Antenatal Care",
                       "Others"],
    "Treatment of Minor Ailments": ["Treatment of Minor Ailments"],
    "Group Contact Activities": ["Number of Adolescents and Young People reached through health talks held at the centre",
                                 "Number of School Visits Conducted",
                                 "Number of Adolescents and Young People reached through school visits",
                                 "Number of Outreaches /Group health talk conducted",
                                 "Number of Adolescents and Young People reached through outreaches/group health talks conducted"
                                 ],

    "Status": ["Number of  In School Youth (ISY) that accessed the centre",
               "Number of  Out School Youth (OSY) that accessed the centre",
               "Number of Clients referred",
               ]
}

In [325]:

# melt=df.melt(id_vars=df.columns[:5])
# melt


In [326]:
# melt.shape

In [327]:
# melt['First'] = melt['variable'].str.rsplit("_",expand=True)
# melt['First'] = melt['variable'].str.rsplit('_', expand=True)[3]
# melt.head(10)

In [335]:

# body
dlist = [(key, i) for key,value in body.items() for i in value]    
beta = pd.DataFrame(dlist, columns=['keys','Beta_indicator'])
#main df
melt=df.melt(id_vars=df.columns[:5])
main =melt['variable'].str.rsplit('_', 2,expand=True).join(melt)
main.rename(columns = {0:'indicators', 1:'Age', 2:'Sex'}, inplace = True)
main.replace(r"_", "", regex=True)
main['indicators'] = main['indicators'].replace('_', ' ', regex=True).str.strip()
beta['Beta_indicator'].str.strip()
#merge
main =main.merge(beta, left_on="indicators", right_on='Beta_indicator')
main.drop(columns =['variable','Beta_indicator'], inplace=True)
main = main.reindex(columns=['Facility', 'State', 'LGA', 'Year', 'Month', 'Sex', 'Age','data','indicators', 'value'])
main.head()

,Facility,State,LGA,Year,Month,Sex,Age,data,indicators,value
0,cr Akamkpa General Hospital,Akwa-Ibom,Akamkpa,NaN,January,male,10-14yrs,NaN,Old Clients,78
1,cr Akamkpa General Hospital,Akwa-Ibom,Calabar South,NaN,February,male,10-14yrs,NaN,Old Clients,157
2,cr Akamkpa General Hospital,Akwa-Ibom,Akamkpa,NaN,March,male,10-14yrs,NaN,Old Clients,163
3,cr Akamkpa General Hospital,Akwa-Ibom,Calabar Municipal,NaN,April,male,10-14yrs,NaN,Old Clients,38
4,cr Akamkpa General Hospital,Akwa-Ibom,Ikom,NaN,May,male,10-14yrs,NaN,Old Clients,55


In [336]:
sales_by_hour = main.groupby(by=["indicators", "Age"]).sum().reset_index()
# patient_df_group =patient_df.groupby(['doctor_id','Month_Year','grouped']).sum().reset_index()


In [337]:
fig = px.bar(sales_by_hour,  x=sales_by_hour.index, y="value",
                 color="Age",
                 #  hover_data=['petal_width'],
                 barmode='group')
fig.show()

TypeError: Argument 'x' is a pandas MultiIndex. pandas MultiIndex is not supported by plotly express at the moment.

In [331]:

beta

,keys,Beta_indicator
0,Total Attendance,Old Clients
1,Total Attendance,New Clients
2,Counseling on,Safe Sex
3,Counseling on,STI Prevention
4,Counseling on,Contraceptive Use
5,Counseling on,Drug Abuse
6,Counseling on,Sexual Violence
7,Counseling on,Unplanned Pregnancy
8,Counseling on,Others (specify)
9,HIV Testing and Couseling,HIV Result negative (-ve)


In [332]:
merged_df =main.merge(beta, left_on="indicators", right_on='Beta_indicator')
merged_df.head()

KeyError: 'indicators'